# Analyze Grape Leaf Morhometrics - Goal: leaf area

## Import Libraries

In [2]:
# Standard library imports
import os
import multiprocessing as mp
import argparse
import traceback
import re
# Third-party libraries
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from pyzbar.pyzbar import decode
from roboflow import Roboflow
from inference_sdk import InferenceHTTPClient
# Local application/library specific imports
from plantcv import plantcv as pcv
from plantcv.parallel import WorkflowInputs
from qreader import QReader

In [ ]:
# Read in size ref images
ref1 = cv2.imread("/Volumes/silvas/SilvasLeafScans2024/Large tray size frame.tif", cv2.IMREAD_COLOR)
ref2 = cv2.imread("/Volumes/silvas/SilvasLeafScans2024/X-Large tray size frame.tif", cv2.IMREAD_COLOR)

In [ ]:
# Display the images using open cv - note: you will have to enter any key to close the images
# any other method will crash the kernel
cv2.imshow("ref1", ref1)
cv2.imshow("ref2", ref2)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

## STEP 1: Find the quarters in the image and calculate the average diameter
### Goal: Divide pixels per dimeter by mm per quarter dia (24.26mm) to get pixels per mm

### Option 1: Use Object detection using https://universe.roboflow.com/s1-sowiy/coins-l4wkp/model/7
#### This method currently works succesfully

In [12]:
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="ifIr3Jg8nPSCOsbGEr86"
)

In [ ]:
# If image is too large, shrink it
max_dimension = 2400
height, width = ref1.shape[:2]
if max(height, width) > max_dimension:
    scaling_factor = max_dimension / float(max(height, width))
    new_size = (int(width * scaling_factor), int(height * scaling_factor))
    ref1 = cv2.resize(ref1, new_size, interpolation=cv2.INTER_AREA)
else:
   print("Image is small enough, no need to resize.")

In [14]:
result = CLIENT.infer(ref1, model_id="coins-l4wkp/7")

In [ ]:
classes = set()
for prediction in result['predictions']:
    classes.add(prediction['class'])

# Convert the set to a list if needed
classes_list = list(classes)
print("Detected classes:", classes_list)

In [ ]:
ref1_copy = ref1.copy()
diameters = []
# Iterate through the predictions to find the bounding box for the class "Marker"
for prediction in result['predictions']:
    x_center = int(prediction['x'])
    y_center = int(prediction['y'])
    diameter = int(max(prediction['width'], prediction['height']))
    radius = diameter // 2
    diameters.append(diameter)
    
    # Create a mask for the circle
    mask = np.zeros(ref1_copy.shape[:2], dtype=np.uint8)
    cv2.circle(mask, (x_center, y_center), radius, 255, -1)
    
    # Find the contours of the circle
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw the contours on the image
    cv2.drawContours(ref1_copy, contours, -1, (0, 0, 0), 2)
    radius = int(max(prediction['width'], prediction['height']) / 2)
    
    # Draw a black circle around the marker
    cv2.circle(ref1_copy, (x_center, y_center), radius, (0, 255, 0), 2)

cv2.imshow("ref1", ref1_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### Option 2: Instance segmentation using https://universe.roboflow.com/ut-tyler-cmpe-3301/coin-detector-jcdoq/model/1
#### I could not find an instance seg model that was public on roboflow for this

In [24]:
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="ifIr3Jg8nPSCOsbGEr86"
)

In [ ]:
result = CLIENT.infer(ref1, model_id="coin-detector-jcdoq/1")
result

In [ ]:
classes = set()
for prediction in result['predictions']:
    classes.add(prediction['class'])

# Convert the set to a list if needed
classes_list = list(classes)
print("Detected classes:", classes_list)

In [ ]:
ref1_copy = ref1.copy()
for prediction in result['predictions']:
    if prediction['class'] == 'Dime':
        points = np.array([[p['x'], p['y']] for p in prediction['points']], dtype=np.int32).reshape((-1, 1, 2))

        # Draw the polyline on the image (example image)
        cv2.polylines(ref1, [points], isClosed=True, color=(0, 255, 0), thickness=2)

cv2.imshow("ref1", ref1)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### Option 3, manual select pixels on ref1 and ref2 to get conversion factor
#### I would suggest finding the pixels coordinates yourself and threshing color within that region

### Once avg quarter diameter in pixels is found use the following equation to create a conversion

In [ ]:
# Note, dia is currently unassigned!
pix_per_mm = dia/25.4

## Read in image to be analyzed

In [ ]:
# Find file path name
image_path = "/Users/aja294/Documents/Grape_local/projects/leaf_morphometrics/images/05-08-24_LR_1-8_B.tif"
file_name = os.path.basename(image_path)
file_name = os.path.splitext(file_name)[0]
print(file_name)

In [34]:
# Read in image on file path
image = cv2.imread(image_path, cv2.IMREAD_COLOR)

In [ ]:
# Visually validate image, note image is in BGR, so colors will be inversed when printing in line wil plt.imshow
plt.imshow(image)
height, width = image.shape[:2]
print(f"Image dimensions: {width} x {height}")

## Deploy a roboflow model to identify leaves, test one we already have on hand first
### The first option is object detection (bounding boxes), the second is instance segmentation (smart polygons), test each 

### Object detection, suggestion: create a sliding box for which to thresh images in

In [ ]:
# Object detection
rf = Roboflow(api_key='l6XPyOniqM4Ecq129cpf')
project = rf.workspace().project("grape_leaves")
model = project.version("1").model

In [38]:
# If image is too large, shrink it
max_dimension = 2400
height, width = image.shape[:2]
if max(height, width) > max_dimension:
    scaling_factor = max_dimension / float(max(height, width))
    new_size = (int(width * scaling_factor), int(height * scaling_factor))
    image = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
else:
   print("Image is small enough, no need to resize.")


In [39]:
results = model.predict(image, confidence=40, overlap=30).json()

In [ ]:
classes = set()
for prediction in results['predictions']:
    classes.add(prediction['class'])

# Convert the set to a list if needed
classes_list = list(classes)
print("Detected classes:", classes_list)

In [41]:
for prediction in results['predictions']:
    if prediction['class'] == 'Marker':
        points = np.array([[p['x'], p['y']] for p in prediction['points']], dtype=np.int32).reshape((-1, 1, 2))

        # Draw the polyline on the image (example image)
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

In [ ]:
image_copy = image.copy()
for (x1, y1, x2, y2) in leaves:
    # Apply mask
    marker_region = image_copy[y1:y2, x1:x2]
    # Further processing on marker_region
    cv2.rectangle(image_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image_copy, 'leaf', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.imshow(image_copy)

In [ ]:
### Option 2: Train an image segmention model to detect leaves

In [ ]:
#Instance segmentation
rf = Roboflow(api_key='l6XPyOniqM4Ecq129cpf')
project = rf.workspace().project("morphometric_segmentation")
model = project.version("2").model

In [ ]:
classes = set()
for prediction in results['predictions']:
    classes.add(prediction['class'])

# Convert the set to a list if needed
classes_list = list(classes)
print("Detected classes:", classes_list)

In [ ]:
image_copy = image.copy()
# print the outline of the leaf
for prediction in results['predictions']:
    if prediction['class'] == 'leaf':
        points = np.array([[p['x'], p['y']] for p in prediction['points']], dtype=np.int32).reshape((-1, 1, 2))

        # Draw the polyline on the image (example image)
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)

plt.imshow(image)

### Step 3, once a mask is created for each leaf, calculate mm2 and cm2 and print to a datasheet

In [ ]:
# Note leaf_area is undefined! Assign this value by finding pixels within a given leaf
leaf_mm2 = leaf_area/pix_per_mm^2